<a href="https://colab.research.google.com/github/gyuholee94/BOK_Project/blob/mingyun%2Ftest/bond_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import requests # 웹 상의 데이터를 가져올 때
from bs4 import BeautifulSoup #가져온 데이터에서 필요한 정보를 파싱할 때
from tqdm.notebook import tqdm


In [ ]:
# for page in range(1, 11):
# for page in range(1, 101, 10):
# for page in range(10):
no_page =126

report_link = []
report_yymm = []
for page in range(no_page):
  url = 'https://finance.naver.com/research/debenture_list.nhn'
# 'where=news&query=깡'
  query={
    'page' : page+1
  }
  resp=requests.get(url,query)
  soup= BeautifulSoup(resp.content, 'lxml')
  a_tag=soup.select('tr td.file a')
  b_tag=soup.select('tr td.date')
  c_tag=[]
  for idx, b in enumerate(b_tag):
    if idx%2==0:
      c_tag.append(b)

  for a, c in zip(a_tag,c_tag):
    report_link.append(a['href'])
    report_yymm.append(c.text)

len(report_link)


3759

In [ ]:
!pip3 install pdfminer.six

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from io import StringIO
from io import open

In [ ]:
def convert_pdf_to_txt():
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams=laparams)
    fp = open('{}.pdf', 'rb') # 여기에 저장된 url에 대한 response.content를 넣어줘야 함
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password = password, caching = caching, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
import requests
from bs4 import BeautifulSoup
file_res = requests.get("https://ssl.pstatic.net/imgstock/upload/research/debenture/1595205128882.pdf") # 여기에 url을 for문으로 돌릴거고
with open('{}.pdf', 'wb') as f: # 여기에 pdf 이름을 정하면 {위에 있는 코드에서 {}.pdf에 넣어주면 돼}
  f.write(file_res.content)

In [ ]:
full_pdf=[]
for link, yymm in tqdm(zip(report_link, report_yymm)):
  file_res = requests.get(link)
  with open('{}.pdf', 'wb') as f:
    f.write(file_res.content)
  v=convert_pdf_to_txt()
  t=re.findall('[가-힣]+',v)
  g=''
  for tt in tqdm(t):
    g+=tt+' '
  full_pdf.append([yymm,g])


In [ ]:
import pandas as pd

In [ ]:
F=pd.DataFrame(full_pdf, columns=['YYMMDD','TXT'])